In [31]:
import tensorflow as tf


In [32]:
v=tf.Variable(0.0)
(v+1).numpy()

1.0

In [33]:
v.assign(5)

<tf.Variable 'UnreadVariable' shape=() dtype=float32, numpy=5.0>

In [34]:
v.assign_add(1)
v.read_value()

<tf.Tensor: shape=(), dtype=float32, numpy=6.0>

In [35]:
w=tf.Variable([[1.0]])
with tf.GradientTape() as t: #自动跟踪变量的梯度计算
    loss = w*w


In [36]:
grad=t.gradient(loss,w)
grad

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[2.]], dtype=float32)>

In [37]:
d=tf.constant([[3.0]])
with tf.GradientTape() as t: #自动跟踪变量的梯度计算
    t.watch(d) #常量要用watch
    dloss = d*d

In [38]:
dloss_d = t.gradient(dloss,d)
dloss_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.]], dtype=float32)>

In [39]:
d=tf.constant([[3.0]])
with tf.GradientTape(persistent=True) as t: #持久
    t.watch(d) #常量要用watch
    y = d*d
    z = y*y

dy_d = t.gradient(y,d)
dy_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[6.]], dtype=float32)>

In [40]:
dz_d = t.gradient(z,d)
dz_d

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[108.]], dtype=float32)>

In [41]:
(train_image,train_labels),_=tf.keras.datasets.mnist.load_data()

In [42]:
train_image=tf.expand_dims(train_image,-1)

In [43]:
train_image.shape

TensorShape([60000, 28, 28, 1])

In [44]:
train_image = tf.cast(train_image/255,tf.float32)

In [45]:
train_labels = tf.cast(train_labels,tf.int64)

In [46]:
dataset = tf.data.Dataset.from_tensor_slices((train_image,train_labels))
dataset

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>

In [47]:
dataset = dataset.shuffle(10000).batch(32).repeat(1)
dataset

<RepeatDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [48]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,[3,3],activation='relu',input_shape=(None,None,1)),
    tf.keras.layers.Conv2D(32,[3,3],activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(10)
])

In [49]:
optimizer=tf.keras.optimizers.Adam()

In [50]:
loss_fun=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

标签为顺序编码

sparse_categorical_crossentropy() 需要指定y_true和y_pred

SparseCategoricalCrossentropy() 直接调用，不需要指定y_true和y_pred

标签为独热编码

categorical_crossentropy()


In [51]:
feature,labels = next(iter(dataset))
feature.shape

TensorShape([32, 28, 28, 1])

In [52]:
predictions = model(feature)
predictions.shape

TensorShape([32, 10])

In [53]:
tf.argmax(predictions,axis=1)

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>

In [54]:
labels

<tf.Tensor: shape=(32,), dtype=int64, numpy=
array([3, 7, 7, 7, 0, 5, 8, 5, 2, 0, 6, 0, 7, 6, 1, 0, 1, 1, 0, 7, 7, 4,
       3, 2, 5, 0, 9, 2, 1, 7, 1, 7], dtype=int64)>

In [55]:
def loss(model,x,y):
    y_ = model(x)
    return loss_fun(y,y_)

In [ ]:
def train_step(model,images,labels):
    with tf.GradientTape() as t:
        loss_step = loss(model,images,labels)
    grads = t.gradient(loss_step,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))

In [ ]:
def train():
    for epoch in range(10):
        for (batch,(images,labels)) in enumerate(dataset):
            train_step(model,images,labels)
        print("Epoch{} is finshed".format(epoch))



In [ ]:
# train()

In [19]:
m = tf.keras.metrics.Mean('acc')
m(10)

<tf.Tensor: shape=(), dtype=float32, numpy=10.0>

In [20]:
m(20)

<tf.Tensor: shape=(), dtype=float32, numpy=15.0>

In [22]:
m.result().numpy()

15.0

In [23]:
m([30,40])

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [24]:
m.result()

<tf.Tensor: shape=(), dtype=float32, numpy=25.0>

In [28]:
m.reset_states()
m(1)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [29]:
m(2)
m.result().numpy()

1.5

In [56]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')
a(labels,model(feature))


<tf.Tensor: shape=(), dtype=float32, numpy=0.1875>